In [1]:
%%capture
%load_ext autoreload
%autoreload 2

In [2]:
# region General Imports
import os
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

from model_optim.model_optimizer import ModelOptimizer

# Dataset
from custom_datasets.fatigue_mi import FatigueMI
from custom_datasets.norm_cho import NormCho2017
from custom_datasets.opt_game_mi import OptGameMI
from custom_datasets.opt_std_mi import OptStdMI

tf.random.set_seed(42)
np.random.seed(42)

2024-04-13 16:29:13.946025: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 16:29:13.946083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 16:29:13.947162: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 16:29:13.953536: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 16:29:14.709709: W tensorflow/compiler/tf2

To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
/home/arazzz/anaconda3/envs/moabb_model_optimization_quant/lib/python3.11/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [15]:
MODELS_LIST = [
    "shallow_conv_net",
    "lstm_net",
    "deep_conv_net",
    "eeg_net",
    "lstm_cnn_net",
    "lstm_cnn_net_v2"
]
MODELS_HYPERPARAMS_DICT = {
    "shallow_conv_net": {
        "max_epochs": 25
    },
    "eeg_net": {
        "max_epochs": 25
    },
    "deep_conv_net": {
        "max_epochs": 25
    },
    "lstm_net": {
        "max_epochs": 10
    },
    "lstm_cnn_net": {
        "max_epochs": 10
    },
    "lstm_cnn_net_v2": {},
}

DATASETS_LIST = [
    FatigueMI,
    # NormCho2017,
    # OptGameMI,
    # OptStdMI,
]

In [4]:
# from model_optim.utils import data_generator

# data_generator(
#     dataset=NormCho2017(),
#     subjects=[1],
#     channel_idx=[],
#     sfreq=128,
# )

In [5]:
subject_files = glob.glob("./temp_v2/*/*/model/study_best_trial.npy")
subject_files_data = {}
for subject_file in subject_files:
    subject_files_data[subject_file] = np.load(subject_file, allow_pickle=True).item()


In [6]:
# Sort subject_files_data by subject_file_data.user_attrs['trial_data']['test_accuracy']
sorted_subject_files_data = dict(sorted(subject_files_data.items(), key=lambda item: item[1].user_attrs['trial_data']['test_accuracy'], reverse=True))
sorted_subject_files_data_test_acc = {k: v.user_attrs['trial_data']['test_accuracy'] for k, v in sorted_subject_files_data.items()}
rpprint(sorted_subject_files_data_test_acc)

{}

In [7]:
model_optimizer = ModelOptimizer(
    dataset=FatigueMI(),
    model_name="shallow_conv_net"
)

Original sfreq: 300.0


In [16]:
for dataset in [FatigueMI]:
    for model in ["deep_conv_net"]:
        model_optimizer = ModelOptimizer(
            dataset=dataset(),
            model_name=model
        )
        for subject in [4]:
            max_epochs = MODELS_HYPERPARAMS_DICT[model]["max_epochs"]
            study = model_optimizer.search_best_model(
                subjects = [subject],
                max_iter = 25,
                max_epochs = max_epochs,
                max_stag_count = 10,
                rounds = 1,
                replace_previous_study_for_subjects = True
            )

Original sfreq: 300.0
Found previous study in ./temp/FatigueMI/[4]/c67f6477f55c43b89f616757c38abfe3/model/deep_conv_net_study_best_trial.npy, removing...
Found previous study in ./temp/FatigueMI/[4]/c67f6477f55c43b89f616757c38abfe3/model/deep_conv_net_study.npy, removing...


  0%|          | 0/25 [00:00<?, ?it/s]

Using sfreq: 256; sfreq is None: True; self.original_sfreq: 300.0; sfreq=256
Adding metadata with 3 columns
Epoch 1/25


2024-04-13 16:33:07.347254: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_14/dropout_56/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1/1 [==============================] - ETA: 0s - loss: 1.2798 - accuracy: 0.5294
Epoch 00001: val_accuracy improved from -inf to 0.55556, storing weights.
1/1 [==============================] - 2s 2s/step - loss: 1.2798 - accuracy: 0.5294 - val_loss: 1.1374 - val_accuracy: 0.5556 - lr: 0.0010
Epoch 2/25
1/1 [==============================] - ETA: 0s - loss: 0.6593 - accuracy: 0.6029
Epoch 00002: val_accuracy did not improve
1/1 [==============================] - 0s 57ms/step - loss: 0.6593 - accuracy: 0.6029 - val_loss: 1.1608 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 3/25
1/1 [==============================] - ETA: 0s - loss: 1.4164 - accuracy: 0.5147
Epoch 00003: val_accuracy did not improve
1/1 [==============================] - 0s 54ms/step - loss: 1.4164 - accuracy: 0.5147 - val_loss: 2.6830 - val_accuracy: 0.5556 - lr: 0.0010
Epoch 4/25
1/1 [==============================] - ETA: 0s - loss: 5.2889 - accuracy: 0.4853
Epoch 00004: val_accuracy did not improve
1/1 [=================

2024-04-13 16:34:08.621455: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_27/dropout_108/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1/1 [==============================] - ETA: 0s - loss: 0.9610 - accuracy: 0.5882
Epoch 00001: val_accuracy improved from -inf to 0.50000, storing weights.
1/1 [==============================] - 2s 2s/step - loss: 0.9610 - accuracy: 0.5882 - val_loss: 0.7702 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/25
1/1 [==============================] - ETA: 0s - loss: 0.7712 - accuracy: 0.5735
Epoch 00002: val_accuracy did not improve
1/1 [==============================] - 0s 45ms/step - loss: 0.7712 - accuracy: 0.5735 - val_loss: 3.8611 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 3/25
1/1 [==============================] - ETA: 0s - loss: 5.2506 - accuracy: 0.5147
Epoch 00003: val_accuracy improved from 0.50000 to 0.55556, storing weights.
1/1 [==============================] - 0s 57ms/step - loss: 5.2506 - accuracy: 0.5147 - val_loss: 2.6794 - val_accuracy: 0.5556 - lr: 0.0010
Epoch 4/25
1/1 [==============================] - ETA: 0s - loss: 2.8287 - accuracy: 0.4853
Epoch 00004: val_accuracy did

In [17]:
model_optimizer.get_study_metrics(study).sort_values(by="scores", ascending=True)

,train_acc,test_acc,val_acc,train_val_acc_diff,train_loss,val_loss,train_val_loss_diff,test_loss,scores,channels_selected,sfreq,batch_size,model_name,subjects
19,0.823529,0.818182,0.666667,0.156863,0.426087,0.746527,0.320440,0.510645,0.111411,"[F3, P4, Pz, F7, F8, A2]",300.0,32,deep_conv_net,[4]
12,0.794118,0.818182,0.666667,0.127451,0.465769,0.722489,0.256720,0.528529,0.111461,"[C3, F3, P4, Pz, Fp1, F8, A2]",300.0,32,deep_conv_net,[4]
14,0.794118,0.681818,0.611111,0.183007,0.511425,0.677925,0.166501,0.601170,0.151535,"[F3, P4, Pz, F7, F8, A2]",300.0,64,deep_conv_net,[4]
20,0.823529,0.727273,0.611111,0.212418,0.393579,0.765586,0.372007,0.634555,0.151635,"[C3, F3, P4, Pz, Fp1, F7, F8, A2]",300.0,32,deep_conv_net,[4]
15,0.852941,0.636364,0.555556,0.297386,0.401931,0.719241,0.317310,0.589530,0.197881,"[C3, F3, P4, Pz, Fp1, F8, A2]",300.0,32,deep_conv_net,[4]
10,0.808824,0.681818,0.555556,0.253268,0.475239,0.728204,0.252965,0.585471,0.197881,"[C3, F3, P4, Pz, Fp1, F8, A2]",128.0,32,deep_conv_net,[4]
6,0.838235,0.500000,0.555556,0.282680,0.433113,0.803900,0.370787,4.245121,0.197881,"[F3, P4, Pz, F7, F8, A2, T4]",256.0,96,deep_conv_net,[4]
7,0.823529,0.500000,0.555556,0.267974,0.459921,0.847574,0.387653,1.007001,0.198181,"[P3, Fz, F4, C4, P4, Pz, Fp2, T3, O2, F7, F8, ...",256.0,96,deep_conv_net,[4]
9,0.794118,0.500000,0.777778,0.016340,0.470152,0.643993,0.173841,0.716353,0.299933,"[F3, Fz, F4, C4, P4, Fp1, T3, F7, F8, T6, T4]",256.0,64,deep_conv_net,[4]
24,0.794118,0.500000,0.722222,0.071895,0.450798,0.678867,0.228069,0.766537,0.327461,"[F3, P4, Pz, F7, F8, A2]",300.0,128,deep_conv_net,[4]


In [18]:
rpprint({ k: v for k, v in study.best_trial.params.items() if not k.startswith("channels") })
rprint("test_accuracy =", study.best_trial.user_attrs["trial_data"]["test_accuracy"])
rprint("val_accuracy =", np.max(study.best_trial.user_attrs["trial_data"]["val_accuracy"]))
rprint("channels_selected =", study.best_trial.user_attrs["trial_data"]["channels_selected"])

{'sfreq': None, 'batch_size': 32}

test_accuracy = 0.8181818127632141

val_accuracy = 0.6666666865348816

channels_selected = ['F3' 'P4' 'Pz' 'F7' 'F8' 'A2']

In [19]:
temp_fatigue_mi_studies = glob.glob("./temp/FatigueMI/**/**/model/*_study.npy") + glob.glob("./temp/FatigueMI/**/**/model/shallow_conv_net_study_best_trial.npy")
temp_fatigue_mi_studies_dict = {}
temp_fatigue_mi_studies_file_names_dict = {}

for study_file in temp_fatigue_mi_studies:
    study = np.load(study_file, allow_pickle=True).item()
    subject_number = int(study_file.split("[")[1].split(']')[0])
    model_name = study_file.split("/")[-1].replace("_study.npy", "").replace("shallow_conv_net_study_best_trial.npy", "shallow_conv_net")
    temp_fatigue_mi_studies_dict[f"{subject_number}_{model_name}"] = study
    temp_fatigue_mi_studies_file_names_dict[f"{subject_number}_{model_name}"] = study_file

In [20]:
filtered_study_trials_concat_df = pd.DataFrame()

for subject_model in temp_fatigue_mi_studies_dict:
    study = temp_fatigue_mi_studies_dict[subject_model]
    study_trials_df = model_optimizer.get_study_metrics(study, **{ "default_model_name": "shallow_conv_net", "subjects": [subject_model.split("_")[0]] })
    # Filter: Top 10 best scores -> Max training accuracy -> Minimum difference between training and validation accuracy -> Max test accuracy = best model
    filtered_study_trials_df = study_trials_df.copy()
    filtered_study_trials_df = filtered_study_trials_df.nsmallest(1, 'scores')
    # filtered_study_trials_df = filtered_study_trials_df.nsmallest(5, 'train_val_acc_diff')
    # filtered_study_trials_df = filtered_study_trials_df[filtered_study_trials_df['train_acc'] == max(filtered_study_trials_df['train_acc'])]
    # filtered_study_trials_df = filtered_study_trials_df[filtered_study_trials_df['train_val_acc_diff'] == min(filtered_study_trials_df['train_val_acc_diff'])]
    # filtered_study_trials_df = filtered_study_trials_df[filtered_study_trials_df['test_acc'] == max(filtered_study_trials_df['test_acc'])]
    filtered_study_trials_concat_df = pd.concat([filtered_study_trials_concat_df, filtered_study_trials_df])
display(filtered_study_trials_concat_df)

,train_acc,test_acc,val_acc,train_val_acc_diff,train_loss,val_loss,train_val_loss_diff,test_loss,scores,channels_selected,sfreq,batch_size,model_name,subjects
19,0.838235,0.818182,0.722222,0.116013,1.209507,1.407622,0.198116,1.633958,0.077361,"[C3, Fz, T6, T4]",256.0,192,eeg_net,[9]
7,0.573529,0.545455,0.555556,0.017974,0.812133,0.654432,0.157701,0.673368,0.198031,"[P3, C3, C4, Pz, Fp1, T3, O1, O2, F8, T6]",128.0,256,deep_conv_net,[9]
18,0.867647,0.545455,0.666667,0.200980,84.76915,82.498848,2.270302,90.271187,0.111561,"[F3, Fz, F4, Fp2, T3, T5, F7, A2, T4]",256.0,224,lstm_cnn_net,[9]
15,0.970588,0.545455,0.777778,0.192810,14.35515,14.367508,0.012358,18.714355,0.049883,"[F3, F4, Cz, Pz, Fp1, Fp2, T5, O1, A2, T6]",256.0,128,lstm_cnn_net,[10]
8,0.882353,0.636364,0.722222,0.160131,0.638915,0.867556,0.228641,1.025751,0.077761,"[P3, C3, C4, P4, Cz, Pz, Fp1, Fp2, T3, O1, O2,...",300.0,32,eeg_net,[10]
14,0.661765,0.590909,0.611111,0.050654,0.636553,0.60518,0.031373,0.682946,0.151685,"[P3, C3, F4, C4, P4, Fp2, F7, A2, T4]",300.0,96,deep_conv_net,[10]
0,0.970588,0.454545,0.777778,0.192810,55.781464,51.300117,4.481346,57.484512,0.049683,"[Fz, F4, T5, F7, F8, T4]",256.0,32,lstm_cnn_net,[11]
3,0.970588,0.681818,0.722222,0.248366,0.755424,1.030562,0.275138,1.222213,0.077861,"[P3, C3, F3, Fz, P4, Cz, T3, T5, O1, F7, F8, A...",256.0,256,eeg_net,[11]
1,0.720588,0.363636,0.666667,0.053922,0.614674,0.683233,0.068558,0.750804,0.111461,"[F3, P4, Cz, Pz, T3, T5, A2]",128.0,160,deep_conv_net,[11]
13,0.941176,0.545455,0.611111,0.330065,1.290258,1.403062,0.112804,1.581581,0.151635,"[Fz, F4, P4, Cz, Fp2, O1, O2, T4]",256.0,192,eeg_net,[2]


In [21]:
filtered_study_trials_concat_df.query("model_name == 'deep_conv_net'").sort_values(by="test_acc", ascending=False) #[['test_acc', 'subjects']] #.head(5)

,train_acc,test_acc,val_acc,train_val_acc_diff,train_loss,val_loss,train_val_loss_diff,test_loss,scores,channels_selected,sfreq,batch_size,model_name,subjects
19,0.823529,0.818182,0.666667,0.156863,0.426087,0.746527,0.32044,0.510645,0.111411,"[F3, P4, Pz, F7, F8, A2]",300.0,32,deep_conv_net,[4]
1,0.735294,0.727273,0.666667,0.068627,0.553756,0.69879,0.145035,0.629561,0.111761,"[P3, F3, Fz, F4, P4, Pz, Fp1, Fp2, O1, F7, A2,...",300.0,224,deep_conv_net,[3]
23,0.852941,0.727273,0.888889,0.035948,0.388678,0.429204,0.040526,0.646307,0.262646,"[C3, F3, C4, Cz, Fp2, T6]",128.0,128,deep_conv_net,[12]
2,0.705882,0.681818,0.666667,0.039216,0.572419,0.690558,0.118139,0.673557,0.111661,"[P3, C3, F3, Fz, F4, C4, Cz, Pz, Fp2, O1, T4]",300.0,64,deep_conv_net,[6]
14,0.661765,0.590909,0.611111,0.050654,0.636553,0.60518,0.031373,0.682946,0.151685,"[P3, C3, F4, C4, P4, Fp2, F7, A2, T4]",300.0,96,deep_conv_net,[10]
7,0.573529,0.545455,0.555556,0.017974,0.812133,0.654432,0.157701,0.673368,0.198031,"[P3, C3, C4, Pz, Fp1, T3, O1, O2, F8, T6]",128.0,256,deep_conv_net,[9]
16,0.779412,0.545455,0.611111,0.168301,0.524894,0.662485,0.137591,0.796263,0.151735,"[P3, C3, F3, C4, P4, Cz, Fp2, T5, O1, T6]",300.0,32,deep_conv_net,[1]
8,0.691176,0.500000,0.555556,0.135621,0.593381,0.727684,0.134303,1.073522,0.197881,"[F3, Fz, T3, O1, F7, A2, T6]",300.0,224,deep_conv_net,[2]
7,0.558824,0.500000,0.555556,0.003268,0.851231,0.709207,0.142025,4.306934,0.197881,"[C3, F3, Fp1, O2, F7, F8, T6]",300.0,96,deep_conv_net,[7]
24,0.632353,0.500000,0.611111,0.021242,0.638411,0.681752,0.043341,0.696188,0.151535,"[P3, C3, F3, O2, A2, T4]",128.0,32,deep_conv_net,[5]


In [ ]:
# EEGNet:           4, 9, 6, 10, 12    5, 11
# DeepConvNet:      4, 9, 6, 10, 12    3,
# ShallowConvNet:   4, 9, 6, 10, 12    1, 11

"""
- Re-run:
-- DeepConvNet on subject 11
"""